Hi David, before I forget,  about the PauliOp problem I told you at lunch on Tuesday. This is how I had originally found the error:

`CustomObservables` not parameterized returns an error using pennylane, whereas qiskit works well.

1. Doing:

```

observable = CustomObservable(num_qubits, ["Z", "I"], parameterized=False)
param_obs = []
qnn.evaluate(x_array, [], param_obs, "f")["f"]

```
returns a pennylane error but not a qiskit error.

2. However, doing:

```

observable = CustomObservable(num_qubits, ["Z", "I"], parameterized=True)
param_obs = [(1) for i in range(observable.num_parameters)] # real coefficients
qnn.evaluate(x_array, [], param_obs, "f")["f"]

```

Works fine.

See the two examples below




In [1]:

from squlearn.observables import *
from squlearn.encoding_circuit import QiskitEncodingCircuit, LayeredEncodingCircuit
import numpy as np
from squlearn.qnn.lowlevel_qnn import LowLevelQNN
from squlearn import Executor
from squlearn.observables import CustomObservable
import numpy as np


num_qubits = 1
x_array = np.array([[0.75], 
                    [0.1]])
circuit = LayeredEncodingCircuit(num_qubits, 1, "x", )
circuit.Rx("x") 

observable = CustomObservable(num_qubits, ["Z", "I"], parameterized=True)
param_obs = [(1) for i in range(observable.num_parameters)] # real coefficients
print("Observable \n", observable)

qnn_pennylane = LowLevelQNN(circuit, observable, Executor("pennylane"))
qnn_qiskit = LowLevelQNN(circuit, observable, Executor("qiskit"))


print("Qiskit with real coefficients for the observable ")
print("f ", qnn_qiskit.evaluate(x_array, [], param_obs, "f")["f"])
print("dfdx", qnn_qiskit.evaluate(x_array, [], param_obs, "dfdx")["dfdx"])
print("dfdop", qnn_qiskit.evaluate(x_array, [], param_obs, "dfdop")["dfdop"])

print("Pennylane with real coefficients for the observable") 
print("f", qnn_pennylane.evaluate(x_array, [], param_obs, "f")["f"])
print("dfdx", qnn_pennylane.evaluate(x_array, [], param_obs, "dfdx")["dfdx"])
print("dfdop", qnn_pennylane.evaluate(x_array, [], param_obs, "dfdop")["dfdop"])


Observable 
 SparsePauliOp(['Z', 'I'],
              coeffs=[ParameterExpression(1.0*p[0]), ParameterExpression(1.0*p[1])])
Qiskit with real coefficients for the observable 
f  [1.73168887 1.99500417]
dfdx [[-0.68163876]
 [-0.09983342]]
dfdop [[0.73168887 1.        ]
 [0.99500417 1.        ]]
Pennylane with real coefficients for the observable
f [1.73168887 1.99500417]
dfdx [[-0.68163876]
 [-0.09983342]]
dfdop [[0 1]
 [0 1]]


In [2]:
from squlearn.observables import *
from squlearn.encoding_circuit import QiskitEncodingCircuit, LayeredEncodingCircuit
import numpy as np
from squlearn.qnn.lowlevel_qnn import LowLevelQNN
from squlearn import Executor
from squlearn.observables import CustomObservable
import numpy as np


num_qubits = 1
x_array = np.array([[0.75], 
                    [0.1]])
circuit = LayeredEncodingCircuit(num_qubits, 1, "x", )
circuit.Rx("x") 

observable = CustomObservable(num_qubits, ["Z", "I"], parameterized=False)
param_obs = []
print("Observable \n", observable)

qnn_pennylane = LowLevelQNN(circuit, observable, Executor("pennylane"))
qnn_qiskit = LowLevelQNN(circuit, observable, Executor("qiskit"))


print("Qiskit with real coefficients for the observable ")
print("f ", qnn_qiskit.evaluate(x_array, [], param_obs, "f")["f"])
print("dfdx", qnn_qiskit.evaluate(x_array, [], param_obs, "dfdx")["dfdx"])
print("dfdop", qnn_qiskit.evaluate(x_array, [], param_obs, "dfdop")["dfdop"])

print("Pennylane with real coefficients for the observable") 
print("f", qnn_pennylane.evaluate(x_array, [], param_obs, "f")["f"])
print("dfdx", qnn_pennylane.evaluate(x_array, [], param_obs, "dfdx")["dfdx"])
print("dfdop", qnn_pennylane.evaluate(x_array, [], param_obs, "dfdop")["dfdop"])


Observable 
 SparsePauliOp(['Z', 'I'],
              coeffs=[1.+0.j, 1.+0.j])
Qiskit with real coefficients for the observable 
f  [1.73168887 1.99500417]
dfdx [[-0.68163876]
 [-0.09983342]]
dfdop []
Pennylane with real coefficients for the observable


TypeError: unsupported operand type(s) for +: 'ExpectationMP' and 'ExpectationMP'